In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import requests
import time
import sys
import os
path=r"F:\Diverse\statistics\Python_data_analysis\2017_data_news\非物质文化遗产"
os.chdir(path)
#plotly离线绘图时必须要初始化操作，po.init_notebook_mode()
#os模块提供系统文件搜索路径，可使用os.chdir()-改变路径,os.path.join()-组合为绝对路径
#sys.path.append(path)
# requests提供url访问服务
# sys提供环境变量搜索路径

### 1.基础数据导入

In [235]:
#导入非物质文化遗产信息
Cul_heritage=pd.read_excel(r"非遗名录-各批次.xlsx",list(range(0,7)))

In [236]:
Cul_her=pd.concat([Cul_heritage[0],Cul_heritage[1],Cul_heritage[2],Cul_heritage[3],Cul_heritage[4],Cul_heritage[5]
                  ,Cul_heritage[6]],ignore_index=True)

In [237]:
Cul_her.head()

,时间,项目名称,申报地区或单位,民族
0,2006.0,春节,中央文化部,NaN
1,2006.0,清明节,中央文化部,NaN
2,2006.0,端午节,中央文化部,NaN
3,2006.0,端午节（屈原故里端午习俗）,湖北省宜昌市秭归县,NaN
4,2006.0,端午节（西塞神舟会）,湖北省黄石市,NaN


In [238]:
Cul_her.shape

(424, 4)

In [239]:
Cul_her.index

RangeIndex(start=0, stop=424, step=1)

### 2.时间数据处理

In [240]:
#datetime.datetime类型用于保存时间对象
#默认时间格式：%Y-%m-%d
#时间对象转化为字符串，str()/strftime("%Y-%m-%d")方法
#字符串转化为时间对象，datetime.strptime(value,"%Y-%m-%d")/pd.to_datetime(value)方法
from datetime import datetime
time=Cul_her["时间"].astype("str")
time=[itime.split(".")[0]+"年" for itime in time]
Cul_her["时间"]=time
#字符串转化为时间对象
#pdtime=pd.to_datetime(date[0])
#strptime=datetime.strptime(date[0].split(" ")[0],"%m/%d/%Y")#需要指定字符串的时间格式

### 3.位置定位

In [142]:
#坐标变换为高德坐标
def transform(location):
    parameters = {'coordsys':'gps','locations': location, 'key': 'a65d57cafeb85818e82da28826ffc240'}
    base = 'http://restapi.amap.com/v3/assistant/coordinate/convert'
    response = requests.get(base, parameters)
    answer = response.json()
    return answer['locations']

#地理坐标定位地理地址
def reverse_geocode(location):
    parameters = {'location': location, 'key': 'a65d57cafeb85818e82da28826ffc240'}
    base = 'http://restapi.amap.com/v3/geocode/regeo'
    response = requests.get(base, parameters)
    answer = response.json()
    return answer['regeocode']['addressComponent']

#地理地址定位地理坐标
def geocode(location):
    parameters = {'address': location, 'key': 'a65d57cafeb85818e82da28826ffc240'}
    base = 'http://restapi.amap.com/v3/geocode/geo'
    response = requests.get(base, parameters)
    answer = response.json()
    return answer['geocodes']


In [241]:
#初始化列表
province=[]
city=[]
district=[]
township=[]
location=[]

In [242]:
#各项目经纬度信息
locations=Cul_her.iloc[:,2]
for ix in range(0,Cul_her.shape[0]):
    ilocation=locations.loc[ix]  
    address=geocode(ilocation)#地址定位坐标
    #装载位置信息:省-市-县-镇-经纬度
    try:
        province.append(address[0]["province"])
        city.append(address[0]["city"])
        district.append(address[0]["district"])
        township.append(address[0]["township"])
        location.append(address[0]["location"])
    except:
        province.append(np.nan)
        city.append(np.nan)
        district.append(np.nan)
        township.append(np.nan)
        location.append(np.nan)

ConnectionError: HTTPConnectionPool(host='restapi.amap.com', port=80): Max retries exceeded with url: /v3/geocode/geo?address=%E4%B8%AD%E5%A4%AE%E6%96%87%E5%8C%96%E9%83%A8&key=a65d57cafeb85818e82da28826ffc240 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x000000000C2AFBE0>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',))

In [171]:
#将地址写入数据表
Cul_her["province"]=province
Cul_her["city"]=city

Cul_her["district"]=district
Cul_her["township"]=township

Cul_her["location"]=location

In [233]:
Cul_her.head()

,时间,项目名称,申报地区或单位,民族,shijian
0,2006.0,春节,中央文化部,NaN,2006年
1,2006.0,清明节,中央文化部,NaN,2006年
2,2006.0,端午节,中央文化部,NaN,2006年
3,2006.0,端午节（屈原故里端午习俗）,湖北省宜昌市秭归县,NaN,2006年
4,2006.0,端午节（西塞神舟会）,湖北省黄石市,NaN,2006年


In [ ]:
Cul_her.to_excel("非遗名录-all.xlsx","all",index=False)